You are now ready to use Python and SQLAlchemy to do basic climate analysis and data exploration on your new weather station tables. All of the following analysis should be completed using SQLAlchemy ORM queries, Pandas, and Matplotlib.


* Create a Jupyter Notebook file called climate_analysis.ipynb and use it to complete your climate analysis and data exporation.
* Choose a start date and end date for your trip. Make sure that your vacation range is approximately 3-15 days total.
* Use SQLAlchemy create_engine to connect to your sqlite database.
* Use SQLAlchemy automap_base() to reflect your tables into classes and save a reference to those classes called Station and Measurement.

In [ ]:
#SQL Alchemy dependencies
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.ext.automap import automap_base

#Other SQLAlchemy dependencies
from sqlalchemy import Column, Integer, String, Float, Date
from sqlalchemy.orm import Session

#dependencies
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
#create an engine an instantiate a session to the sqlite database
engine = create_engine('sqlite:///hawaii_db.sqlite')
session = Session(engine)
inspector = inspect(engine)

In [ ]:
#test connection to the database with a simple query
engine.execute('SELECT * FROM measurement LIMIT 10').fetchall()

In [ ]:
#attempt to map classes from SQL database
Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
#Attempt is not successful
Base.classes.keys()

In [ ]:
#Attempt is not successful
Base.classes.items()

## Precipitation Analysis


* Design a query to retrieve the last 12 months of precipitation data.
* Select only the date and prcp values.
* Load the query results into a Pandas DataFrame and set the index to the date column.
* Plot the results using the DataFrame plot method.

In [ ]:
#Create a dataframe that holds all the precipitation measurements for the last 12 months of the dataset
measurement_df = pd.read_sql('measurement', engine, index_col = 'id')
measurement_df_TTM = measurement_df[measurement_df['date'] > "2016-08-23"]
precipitation_df = measurement_df_TTM[['date', 'prcp']]

In [ ]:
#Graph precipitation over the past 12 months
precipitation_df.plot(x = 'date', y = 'prcp', label = "Precipitation", figsize = (10,5), ylim = (0,7), title = "Precipitation over the last 12 months")

## Station Analysis


* Design a query to calculate the total number of stations.
* Design a query to find the most active stations.
* * List the stations and observation counts in descending order
* * Which station has the highest number of observations?
* Design a query to retrieve the last 12 months of temperature observation data (tobs).
* * Filter by the station with the highest number of observations.
* * Plot the results as a histogram with bins=12.

In [ ]:
#find number of stations
tot_stations = measurement_df['station'].nunique()
print(f"Total number of stations is {tot_stations}")

In [ ]:
#find most active stations
stations_df = measurement_df[['station', 'date']].groupby('station').count().sort_values(by = 'date', ascending = False)
stations_df = stations_df.rename(columns = {'date': 'Observation_Count'})
stations_df

In [ ]:
#filter to the most active station & plot a histogram to display that data
most_obs_id = measurement_df_TTM[['date', 'station', 'tobs']].groupby('station').count().sort_values(by = "tobs", ascending = False).index[0]
most_obs = measurement_df_TTM[measurement_df_TTM['station'] == most_obs_id]
most_obs.plot(x = 'date', y = 'tobs', kind = "hist", bins = 12, title = "temperature observation data (tobs)")

Write a function called calc_temps that will accept a start date and end date in the format %Y-%m-%d and return the minimum, average, and maximum temperatures for that range of dates.

* Use the calc_temps function to calculate the min, avg, and max temperatures for your trip using the matching dates from the previous year (i.e. use "2017-01-01" if your trip start date was "2018-01-01")
* Plot the min, avg, and max temperature from your previous query as a bar chart.
* Use the average temperature as the bar height.
* Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr).

In [ ]:
#convert the date text to a datetime object
measurement_df['date'] = measurement_df['date'].apply(pd.to_datetime)

In [ ]:
#find the latest date in the dataset
latest = measurement_df['date'].max() + timedelta(365)

In [ ]:
#define the input variables
start = input("Enter the start date for your trip?  (must be in the format YYYY-MM-DD)")
end = input(f"Enter the end date for your trip?  (must be in the format YYYY-MM-DD and before {latest.strftime('%Y-%m-%d')})")

def calc_temps(start, end):
    converted_start = datetime.strptime(start, '%Y-%m-%d')
    comp_start = converted_start - timedelta(365)
    converted_end = datetime.strptime(end, '%Y-%m-%d')
    comp_end = converted_end - timedelta(365)
    if converted_start < converted_end and converted_end < latest:
        temps = measurement_df[(measurement_df['date'] <= comp_end) & (measurement_df['date'] >= comp_start)]
        temps['temp'] = 'Temperature (F)'
        temps_for_trip = temps[['tobs', 'temp']]
        temps_for_trip = temps_for_trip.rename(columns = {'tobs': 'Temp Obs (f)'})
    else:
        print("You've entered incorrect trip dates.  Please try again.")
    return temps_for_trip, comp_start, comp_end
         
trip_temp_for_graph, comp_start, comp_end = calc_temps(start, end)

In [ ]:
sns.barplot(data = trip_temp_for_graph, ci = 'sd')
plt.ylim(70, 82)
plt.ylabel('Average Temperature (f)')
plt.title(f"Average Temperatures from {comp_start.strftime('%Y-%m-%d')} to {comp_end.strftime('%Y-%m-%d')}")